In [13]:
from pathlib import Path
import torch
from fastai.data.transforms import get_image_files
from torch.utils.data.dataset import Dataset
from torchvision.io import read_image

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

class ImageDataset(Dataset):
    def __init__(
        self, folder: Path, recurse=True, transform=None, target_transform=None
    ):
        paths = get_image_files(folder, recurse=recurse)
        labels = [int(p.parent.name) for p in paths]
        self.ds = list(zip(paths, labels))
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        sample = self.ds[idx]
        img = read_image(str(sample[0])).float() / 255
        label = sample[1]
        if self.transform:
            img = self.transform(img)
        if self.target_transform:
            label = self.target_transform(label)
        return img, label

In [14]:
from torchvision.transforms.v2 import Lambda
from torch.utils.data import DataLoader


train_ds = ImageDataset(
    Path("MNIST/mnist_png/training"),
    target_transform=Lambda(
        lambda y: torch.zeros(10, dtype=torch.float).scatter_(
            dim=0, index=torch.tensor(y), value=1
        )
    ),
)
test_ds = ImageDataset(Path("MNIST/mnist_png/testing"))

train_dl = DataLoader(train_ds, 256, True)
test_dl = DataLoader(test_ds)

In [15]:
# import matplotlib.pyplot as plt
# x,y = next(iter(train_dl))
# fig, axes = plt.subplots(2, 4, figsize=(12, 6))
# axes = axes.flatten()

# for i in range(8):
#     img = x[i].permute(1, 2, 0).squeeze()
#     label = y[i]
#     axes[i].imshow(img, cmap="gray")
#     axes[i].set_title(f"Label: {label}")
#     axes[i].axis("off")

# plt.tight_layout()
# plt.show()

In [16]:
import torch.nn as nn


class SadPepeNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.hidden_layers = nn.Sequential(
            nn.Linear(28 * 28, 30),
            nn.ReLU(),
            nn.Linear(30, 10),
        )

    def forward(self, X):
        input = self.flatten(X)
        output = self.hidden_layers(input)
        return output

In [17]:
model = SadPepeNet()

In [18]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [19]:
def test():
    model.eval()
    correct = 0
    with torch.no_grad():
        for x, y in test_dl:
            l = model(x.to(device))
            probs = nn.Softmax(dim=1)(l)
            pred = probs.argmax(1)
            correct += (pred == y.to(device)).float().sum().item()
        correct /= len(test_dl.dataset)
    print(f"accuracy: {correct*100}٪")

In [20]:
for epoch in range(1, 5):
    model.train()
    print(f"epoch: {epoch}")
    for batch, (X, y) in enumerate(train_dl):
        # Compute prediction and loss
        logits = model(X.to(device))
        loss = loss_fn(logits, y.to(device))

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * 256 + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{len(train_ds):>5d}]")
    test()

epoch: 1
loss: 2.298727  [  256/60000]
loss: 2.291258  [25856/60000]


KeyboardInterrupt: 